In [5]:
import sys, os, git

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/python-codes/")

from InitializeSimulation import InitializeSimulation
from Utilities import Utilities
from Outputs import Outputs
from MolecularDynamics import MolecularDynamics
from MonteCarlo import MonteCarlo

import numpy as np

In [10]:
self = MolecularDynamics(number_atoms=[500, 10],
                      epsilon=[0.1, 0.1], # kcal/mol
                      sigma=[2, 1], # A
                      atom_mass= [1, 1], # g/mol
                      Lx=30, # A
                      Ly=30, # A
                      Lz=30, # A
                      minimization_steps = 1,
                      maximum_steps=1,
                      desired_temperature=300,
                      thermo = 10,
                      dump = 10,  
                      tau_temp = 100, # fs
                      #tau_press= 1000, # fs
                      time_step=1, # fs
                      seed=319817,
                      data_folder = "mdcode-output/",
                      )
self.run()
# run lammps for comparison
os.system("/home/simon/Softwares/lammps-2Aug2023/src/lmp_serial -in input.lammps")

step epot maxF
0 81578718.352 79181512703.378
 step     N  temp  epot  ekin press   vol
    0   510 7419910426950.55 14456.463885080151 11257714046966.023 19059844553793.42 27000.0
LAMMPS (2 Aug 2023 - Update 1)
Reading data file ...
  orthogonal box = (-15 -15 -15) to (15 15 15)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  510 atoms
  reading velocities ...
  510 velocities
  read_data CPU = 0.006 seconds
Generated 1 of 1 mixed pair_coeff terms from geometric mixing rule
Neighbor list info ...
  update: every = 1 steps, delay = 0 steps, check = yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 12
  ghost atom cutoff = 12
  binsize = 6, bins = 5 5 5
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair lj/cut, perpetual
      attributes: half, newton on
      pair build: half/bin/atomonly/newton
      stencil: half/bin/3d
      bin: standard
Setting up cg style minimization ...
  Unit style    : real
  Current step  : 0
Per MPI rank

256

In [11]:
import time

In [69]:
ti = time.time()

forces = np.zeros((self.total_number_atoms,3))
box_size = np.diff(self.box_boundaries).reshape(3)
for Ni in range(self.total_number_atoms-1):
    position_i = self.atoms_positions[Ni]
    sigma_i = self.atoms_sigma[Ni]
    epsilon_i = self.atoms_epsilon[Ni]
    for Nj in np.arange(Ni+1,self.total_number_atoms):
        position_j = self.atoms_positions[Nj]
        rij_xyz = (np.remainder(position_i - position_j + box_size/2., box_size) - box_size/2.).T
        rij = np.sqrt(np.sum(rij_xyz**2))
        if rij < self.cut_off:
            sigma_j = self.atoms_sigma[Nj]
            epsilon_j = self.atoms_epsilon[Nj]
            sigma_ij = (sigma_i+sigma_j)/2
            epsilon_ij = (epsilon_i+epsilon_j)/2
            dU_dr = 48*epsilon_ij/rij*((sigma_ij/rij)**12-0.5*(sigma_ij/rij)**6)
            forces[Ni] += dU_dr*rij_xyz/rij
            forces[Nj] -= dU_dr*rij_xyz/rij

tf = time.time()
print("duration =", np.round(tf-ti,3), "s")

duration = 2.388 s


In [68]:
ti = time.time()

forces = np.zeros((self.total_number_atoms,3))
box_size = np.diff(self.box_boundaries).reshape(3)

for Ni, position_i, sigma_i, epsilon_i in zip(np.arange(self.total_number_atoms-1),
                                              self.atoms_positions,
                                              self.atoms_sigma,
                                              self.atoms_epsilon):
    positions_j = self.atoms_positions[Ni+1:]
    sigma_j = self.atoms_sigma[Ni+1:]
    epsilon_j = self.atoms_epsilon[Ni+1:]
    rij_xyz = (np.remainder(position_i - positions_j + box_size/2., box_size) - box_size/2.).T
    rij = np.linalg.norm(rij_xyz, axis=0)
    for Nj, sigma_j0, epsilon_j0, rij0, rij_xyz0 in zip(np.arange(Ni+1,self.total_number_atoms)[rij < self.cut_off],
                                      sigma_j[rij < self.cut_off], epsilon_j[rij < self.cut_off],
                                      rij[rij < self.cut_off], rij_xyz.T[rij < self.cut_off]):
        sigma_ij = (sigma_i+sigma_j0)/2
        epsilon_ij = (epsilon_i+epsilon_j0)/2
        dU_dr = 48*epsilon_ij/rij0*((sigma_ij/rij0)**12-0.5*(sigma_ij/rij0)**6)
        forces[Ni] += dU_dr*rij_xyz0/rij0
        forces[Nj] -= dU_dr*rij_xyz0/rij0

tf = time.time()
print("duration =", np.round(tf-ti,3), "s")

duration = 0.358 s
